# 1. 论文引用网络节点分类比赛


## 1.1 赛题介绍


图神经网络（Graph Neural Network）是一种专门处理图结构数据的神经网络，目前被广泛应用于推荐系统、金融风控、生物计算中。图神经网络的经典问题主要有三种，包括节点分类、连接预测和图分类三种。本次比赛是图神经网络7日打卡课程的大作业，主要让同学们熟悉如何图神经网络处理节点分类问题。

数据集为论文引用网络，图由大量的学术论文组成，节点之间的边是论文的引用关系，每一个节点提供简单的词向量组合的节点特征。我们的目的是给每篇论文推断出它的论文类别。

## 1.2 解题思路


本次比赛在Baseline的基础上使用了修改过后的GAT网络结构。修改的地方主要是在GAT的基础上增加了残差的思想，并命名为ResGAT。该思路借鉴了百度的图神经网络课程“图神经网络7日打卡营”中课节6中的内容。并在此基础上进行了调参。并加大训练次数，提交后目前分数可以到0.7519。主要经验有两个：
1. 调参
2. 加大训练次数

## 1.3 运行方式
本次基线基于飞桨PaddlePaddle 1.8.4版本，若本地运行则可能需要额外安装pgl、easydict、pandas等模块。

### 1.3.1 本地运行
下载左侧文件夹中的所有py文件（包括build_model.py, model.py）,以及work目录，然后在右上角“文件”->“导出Notebook到py”，这样可以保证代码是最新版本），执行导出的py文件即可。完成后下载submission.csv提交结果即可。

### 1.3.2 AI Studio (Notebook)运行
依次运行下方的cell，完成后下载submission.csv提交结果即可。若运行时修改了cell，推荐在右上角重启执行器后再以此运行，避免因内存未清空而产生报错。 Tips：若修改了左侧文件夹中数据，也需要重启执行器后才会加载新文件。

In [1]:
!pip install easydict -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple


In [2]:
!pip install paddlepaddle-gpu==1.8.4.post97 -f https://paddlepaddle.org.cn/whl/stable.html

Looking in indexes: https://mirror.baidu.com/pypi/simple/
Looking in links: https://paddlepaddle.org.cn/whl/stable.html


In [3]:
!pip install pgl==1.2.0 -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple


## 1.4 代码整体逻辑

1. 读取提供的数据集，包含构图以及读取节点特征（用户可自己改动边的构造方式）

2. 配置化生成模型，用户也可以根据教程进行图神经网络的实现。

3. 开始训练

4. 执行预测并产生结果文件


## 1.5 环境配置

该项目依赖飞桨paddlepaddle==1.8.4, 以及pgl==1.2.0。请按照版本号下载对应版本就可运行。

In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code,
# so that every time the environment (kernel) starts,
# just run the following code:
import paddle.fluid as F
import sys
sys.path.append('/home/aistudio/external-libraries')

# 2. 具体代码

## 2.1 加载必要的库

In [5]:
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd
import os

## 2.2 图网络配置

In [6]:
from easydict import EasyDict as edict
'''
config = {
    "model_name": "ResGAT",
    "num_layers": 2,
    "dropout": 0.2,
    "learning_rate": 0.001,
    "weight_decay": 0.0005,
    "edge_dropout": 0.1,
    "epoch": 10000
}'''
config = {
    "model_name": "res_unimp_large",
    "num_layers": 3,
    "hidden_size": 128,
    "heads": 2,
    "learning_rate": 0.001,
    "dropout": 0.5,
    "weight_decay": 0.0005,
    "edge_dropout": 0.5,
    "use_label_e": True
}


config = edict(config)

## 2.3 数据加载模块

这里主要是用于读取数据集，包括读取图数据构图，以及训练集的划分。

In [7]:
from collections import namedtuple

Dataset = namedtuple("Dataset",
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index"])

def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("work/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])

    return edges

def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("work/feat.npy")
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})

    indegree = graph.indegree()
    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)

    df = pd.read_csv("work/train.csv")
    node_index = df["nid"].values
    node_label = df["label"].values
    train_part = int(len(node_index) * 0.8)
    train_index = node_index[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_index[train_part:]
    valid_label = node_label[train_part:]
    test_index = pd.read_csv("work/test.csv")["nid"].values
    dataset = Dataset(graph=graph,
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index, num_classes=35)
    # print(len(train_index))
    return dataset

In [8]:
dataset = load()

train_index = dataset.train_index
train_label = np.reshape(dataset.train_label, [-1 , 1])
train_index = np.expand_dims(train_index, -1)

val_index = dataset.valid_index
val_label = np.reshape(dataset.valid_label, [-1, 1])
val_index = np.expand_dims(val_index, -1)

test_index = dataset.test_index
test_index = np.expand_dims(test_index, -1)
test_label = np.zeros((len(test_index), 1), dtype="int64")

## 2.4 组网模块

这里是组网模块，使用了model.py文件中的ResGAT网络。

In [9]:
import pgl
import model

import paddle.fluid as fluid
import numpy as np
import time
from build_model import build_model

place = fluid.CUDAPlace(0)

train_program = fluid.default_main_program()
startup_program = fluid.default_startup_program()
with fluid.program_guard(train_program, startup_program):
    with fluid.unique_name.guard():
        gw, loss, acc, pred = build_model(dataset,
                            config=config,
                            phase="train",
                            main_prog=train_program)

test_program = fluid.Program()
with fluid.program_guard(test_program, startup_program):
    with fluid.unique_name.guard():
        _gw, v_loss, v_acc, v_pred = build_model(dataset,
            config=config,
            phase="test",
            main_prog=test_program)


test_program = test_program.clone(for_test=True)

exe = fluid.Executor(place)

## 2.5 开始训练过程

图神经网络采用FullBatch的训练方式，每一步训练就会把所有整张图训练样本全部训练一遍。



In [10]:
epoch = 10000
exe.run(startup_program)

# 将图数据变成 feed_dict 用于传入Paddle Excecutor
feed_dict = gw.to_feed(dataset.graph)

# for early stopping
count = 0

save_path='resgat_model'
if os.path.exists(save_path):
	print("使用持久化变量模型作为预训练模型")
	fluid.io.load_persistables(executor=exe,dirname=save_path)

for epoch in range(epoch):
    # Full Batch 训练
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(train_index, dtype="int64")
    feed_dict["node_label"] = np.array(train_label, dtype="int64")

    train_loss, train_acc = exe.run(train_program,
                                feed=feed_dict,
                                fetch_list=[loss, acc],
                                return_numpy=True)

    # Full Batch 验证
    # 设定图上面那些节点要获取
    # node_index: 训练节点的nid
    # node_label: 训练节点对应的标签
    feed_dict["node_index"] = np.array(val_index, dtype="int64")
    feed_dict["node_label"] = np.array(val_label, dtype="int64")
    val_loss, val_acc = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_loss, v_acc],
                            return_numpy=True)
    if epoch % 10 == 0:
        print("Epoch", epoch, "Train Acc", train_acc[0], "Valid Acc", val_acc[0])
    if val_acc[0] > 0.75:
        count += 1
    else:
        count = 0
    if count > 2:
        os.makedirs(save_path)
        # 保存持久化变量模型
        fluid.io.save_persistables(executor=exe, dirname=save_path)
        break

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:1070: UserWarning: The following exception is not an EOF exception.
  "The following exception is not an EOF exception.")


EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::string paddle::platform::GetTraceBackString<std::string const&>(std::string const&, char const*, int)
1   paddle::platform::EnforceNotMet::EnforceNotMet(std::string const&, char const*, int)
2   paddle::framework::OperatorWithKernel::ParseInputDataType(paddle::framework::ExecutionContext const&, std::string const&, paddle::framework::proto::VarType_Type*) const
3   paddle::framework::OperatorWithKernel::IndicateVarDataType(paddle::framework::ExecutionContext const&, std::string const&) const
4   paddle::operators::ReshapeOp::GetExpectedKernelType(paddle::framework::ExecutionContext const&) const
5   paddle::framework::OperatorWithKernel::ChooseKernel(paddle::framework::RuntimeContext const&, paddle::framework::Scope const&, paddle::platform::Place const&) const
6   paddle::framework::OperatorWithKernel::RunImpl(paddle::framewor

## 2.6 对测试集进行预测

训练完成后，我们对测试集进行预测。预测的时候，由于不知道测试集合的标签，我们随意给一些测试label。最终我们获得测试数据的预测结果。


In [ ]:
exe.run(startup_program)
feed_dict = gw.to_feed(dataset.graph)

In [ ]:
save_path='resgat_model'
if os.path.exists(save_path):
	print("加载训练好的模型")
	fluid.io.load_persistables(executor=exe,dirname=save_path)

In [ ]:
feed_dict["node_index"] = np.array(test_index, dtype="int64")
feed_dict["node_label"] = np.array(test_label, dtype="int64") #假标签
test_prediction = exe.run(test_program,
                            feed=feed_dict,
                            fetch_list=[v_pred],
                            return_numpy=True)[0]

## 2.7 生成提交文件

最后一步，我们可以使用pandas轻松生成提交文件，最后下载 submission.csv 提交就好了。

In [ ]:
submission = pd.DataFrame(data={
                            "nid": test_index.reshape(-1),
                            "label": test_prediction.reshape(-1)
                        })
submission.to_csv("submission.csv", index=False)

# 3. 总结
本次比赛参加过后有如下几点心得：

1. 百度提供的比赛很赞，即收获了知识又增长了经验，还有可能获得奖品。
2. 图神经网络跟传统的卷积网络等相比有一定的难度，但是有了baseline一切就变得简单起来。
3. 可以结合其它深度学习知识，对网络进行结构上的修改。
4. 多调参和无脑增加训练次数有时总会收到意想不到的收获。
5. 后续可以采用模型融合的方式等进一步提高分数。